In [2]:
import minsearch
import json
from openai import OpenAI
from dotenv import dotenv_values

In [3]:
API_KEY = dotenv_values(".env")["API_KEY"]

In [4]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
    )

In [8]:
q = "The course has already started, can i enroll now ?"

In [9]:
index.fit(documents)

In [10]:
boost = {"question": 3.0, "section": 0.5, }
results = index.search(
    query = q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict= boost, 
    num_results=5,
    
)

In [11]:
results

[{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question':

In [12]:
client = OpenAI(api_key=API_KEY, base_url="https://api.perplexity.ai")

In [13]:
response = client.chat.completions.create(
    model = "llama-3-sonar-small-32k-chat",
    messages = [{
        "role": "user",
        "content": q
    }]
)

In [14]:
response.choices[0].message.content

"I'm happy to help you with your question However, I need a bit more context to provide a accurate answer.\n\nCan you please tell me more about the course you're interested in? What is the course about, and who is the provider (e.g. university, online platform, etc.)? Additionally, what is the current date and how far along is the course in terms of weeks or modules?\n\nThis information will help me better understand your situation and provide a more informed response."

In [23]:
propmt_template = """"You're a course teaching assistant. Answer the QUESTION based on CONTEXT. 
Use only the facts from CONTEXT when answering the QUESTION.
If the CONTEXT doesnt contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}""".strip() ## specify the role # prompt engineering 

In [24]:
context = ""

for doc in results:
    context = context + f"section : {doc['section']} \nquestion: {doc['question']} \nanswer: {doc['text']}\n\n"

In [25]:
print(context)

section : General course-related questions 
question: Course - Can I follow the course after it finishes? 
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section : General course-related questions 
question: Course - When will the course start? 
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

section : General course-related ques

In [26]:
prompt = propmt_template.format(question=q, context = context)

In [27]:
print(prompt)

"You're a course teaching assistant. Answer the QUESTION based on CONTEXT. 
Use only the facts from CONTEXT when answering the QUESTION.
If the CONTEXT doesnt contain the answer, output NONE.

QUESTION: The course has already started, can i enroll now ?

CONTEXT: section : General course-related questions 
question: Course - Can I follow the course after it finishes? 
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section : General course-related questions 
question: Course - When will the course start? 
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course publ

In [34]:
# putting this to gpt:
response = client.chat.completions.create(
    model = "llama-3-sonar-large-32k-chat",
    messages = [{"role": 'user', "content": prompt}],
)

print(response.choices[0].message.content)

Yes.
